# Libraries

In [2]:
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt
from IPython.display import display
import holidays

In [3]:
india_holidays = holidays.India()

# Utils

In [45]:
def read_file(path,dropna=True):
    p = path.replace('\\','/')
    df = pd.read_excel(p)
    if dropna:
        df.dropna(inplace=True)
    display(df.head())
    display(df.info())
    display('The shape of the DataFrame is:',df.shape)
    return df

def dropper(df,col_name):
    df_temp = df.drop(col_name,axis=1)
    return df_temp

def journey(df):
    df_temp = df.copy(deep=True)
    india_holidays = holidays.India()
    df_temp[['Date_of_Journey']]=pd.to_datetime(df_temp['Date_of_Journey'])
    df_temp['Holiday']=(df_temp['Date_of_Journey'].isin(india_holidays)).astype(int)
    
    if len(list(df_temp['Holiday'].unique()))<2:
        df_temp = dropper(df_temp,'Holiday')
        print('There are no Holidays!!!')
        
    df_temp['Journey Date']=df_temp['Date_of_Journey'].dt.day
    df_temp['Journey Month']=df_temp['Date_of_Journey'].dt.month
    df_temp['Journey Year']=df_temp['Date_of_Journey'].dt.year
    df_temp['Journey Day'] = df_temp['Date_of_Journey'].dt.dayofweek
    df_temp['Journey Weekend'] = np.where(((df_temp['Date_of_Journey']).dt.dayofweek) > 5,1,0)
    df_temp['Journey Week']=df_temp['Date_of_Journey'].dt.week
    
    display(df_temp.head())
    
    return df_temp

def durations(df):
    df_temp =df.copy(deep=True)
    duration = list(df_temp.Duration)
    for i in range(len(duration)):
        if len(duration[i].split())!=2:
            if 'h' in duration[i]:
                duration[i] = duration[i].strip()+' 0m'
            else:
                duration[i] = '0h ' + duration[i].strip()

    duration_hour =[]
    duration_min = []

    for i in range(len(duration)):
        duration_hour.append(duration[i].split(sep='h')[0])
        duration_min.append(duration[i].split(sep='m')[0].split()[-1])
        
    df_temp = dropper(df_temp,['Duration'])
   
    df_temp['Duration_hour']=duration_hour
    df_temp['Duration_min']=duration_min
    
    df_temp['Duration_min_scaled'] =df_temp['Duration_min'].astype(float)/60 #converting it into the hours
    df_temp['Duration'] = df_temp['Duration_hour'].astype(float)+df_temp['Duration_min_scaled']
    
    df_temp = dropper(df_temp,['Duration_min_scaled','Duration_hour','Duration_min'])
    
    display(df_temp.head())
    
    return df_temp

def stops(df):
    df_temp = df.copy(deep=True)
    stops = {'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}
    df_temp['Total_Stops']=df_temp['Total_Stops'].map(stops)
    
    display(df_temp.head())
    
    return df_temp

def timing(x):
    if (x > 7) and (x <= 12):
        return 'Morning'
    elif (x > 12) and (x <= 17 ):
        return 'Afternoon'
    elif (x > 17) and (x <= 20):
        return'Evening'
    elif (x > 20) or (x<=7):
        return 'Night'

def times(df):
    df_temp = df.copy(deep=True)
    
    df_temp['Dep_Time']=pd.to_datetime(df_temp['Dep_Time'],format='%H:%M')
    df_temp['Arrival_Time1']=df_temp['Arrival_Time'].apply(lambda x: x.split()[0])
    df_temp['Arrival_Time1']=pd.to_datetime(df_temp['Arrival_Time1'],format='%H:%M')
    
    df_temp['Dep_Duration']=df_temp['Dep_Time'].dt.hour.apply(timing)
    df_temp['Arrival_Duration']=df_temp['Arrival_Time1'].dt.hour.apply(timing)
    
    df_temp=dropper(df_temp,['Arrival_Time1','Dep_Time','Arrival_Time'])
    display(df_temp.head())
    
    return df_temp

# Main

In [31]:
path = os.getcwd()+'\\Dataset\\Data_Train.xlsx'
train_df = read_file(path,True)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10682 non-null  object
 10  Price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1001.4+ KB


None

'The shape of the DataFrame is:'

(10682, 11)

## Date of Journey Column

In [32]:
train_df = journey(train_df)

There are no Holidays!!!


/Users/saad/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019,6,1,12
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,5,1,2019,5,0,1
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,6,9,2019,4,0,36
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,5,12,2019,3,0,49
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,3,1,2019,3,0,1


In [33]:
train_df = dropper(train_df,['Date_of_Journey','Route'])

In [34]:
train_df.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week
0,IndiGo,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019,6,1,12
1,Air India,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,7662,5,1,2019,5,0,1
2,Jet Airways,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,13882,6,9,2019,4,0,36
3,IndiGo,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,6218,5,12,2019,3,0,49
4,IndiGo,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,13302,3,1,2019,3,0,1


## Duration Column

In [35]:
train_df = durations(train_df)

,Airline,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week,Duration
0,IndiGo,Banglore,New Delhi,22:20,01:10 22 Mar,non-stop,No info,3897,24,3,2019,6,1,12,2.833333
1,Air India,Kolkata,Banglore,05:50,13:15,2 stops,No info,7662,5,1,2019,5,0,1,7.416667
2,Jet Airways,Delhi,Cochin,09:25,04:25 10 Jun,2 stops,No info,13882,6,9,2019,4,0,36,19.000000
3,IndiGo,Kolkata,Banglore,18:05,23:30,1 stop,No info,6218,5,12,2019,3,0,49,5.416667
4,IndiGo,Banglore,New Delhi,16:50,21:35,1 stop,No info,13302,3,1,2019,3,0,1,4.750000


## Additional Info & Total Stops columns

In [36]:
train_df['Additional_Info'].value_counts()/train_df.shape[0]

No info                         0.781127
In-flight meal not included     0.185546
No check-in baggage included    0.029957
1 Long layover                  0.001779
Change airports                 0.000655
Business class                  0.000374
No Info                         0.000281
2 Long layover                  0.000094
1 Short layover                 0.000094
Red-eye flight                  0.000094
Name: Additional_Info, dtype: float64

In [37]:
train_df = dropper(train_df,['Additional_Info'])

In [38]:
train_df['Total_Stops'].value_counts()

1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [39]:
train_df = stops(train_df)

,Airline,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week,Duration
0,IndiGo,Banglore,New Delhi,22:20,01:10 22 Mar,0,3897,24,3,2019,6,1,12,2.833333
1,Air India,Kolkata,Banglore,05:50,13:15,2,7662,5,1,2019,5,0,1,7.416667
2,Jet Airways,Delhi,Cochin,09:25,04:25 10 Jun,2,13882,6,9,2019,4,0,36,19.000000
3,IndiGo,Kolkata,Banglore,18:05,23:30,1,6218,5,12,2019,3,0,49,5.416667
4,IndiGo,Banglore,New Delhi,16:50,21:35,1,13302,3,1,2019,3,0,1,4.750000


## Departure and Arrival time

In [47]:
train_df = times(train_df)

,Airline,Source,Destination,Total_Stops,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week,Duration,Dep_Duration,Arrival_Duration
0,IndiGo,Banglore,New Delhi,0,3897,24,3,2019,6,1,12,2.833333,Night,Night
1,Air India,Kolkata,Banglore,2,7662,5,1,2019,5,0,1,7.416667,Night,Afternoon
2,Jet Airways,Delhi,Cochin,2,13882,6,9,2019,4,0,36,19.000000,Morning,Night
3,IndiGo,Kolkata,Banglore,1,6218,5,12,2019,3,0,49,5.416667,Evening,Night
4,IndiGo,Banglore,New Delhi,1,13302,3,1,2019,3,0,1,4.750000,Afternoon,Night


## Categorical Variables

Airline             object
Source              object
Destination         object
Dep_Time            object
Arrival_Time        object
Total_Stops          int64
Price                int64
Journey Date         int64
Journey Month        int64
Journey Year         int64
Journey Day          int64
Journey Weekend      int32
Journey Week         int64
Duration           float64
Arrival_Time1       object
dtype: object

In [ ]:



pd.to_datetime(foo['Arrival_Time'],format='%H:%M').dt.time

In [21]:
foo = train_df.copy(deep=True)
foo.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week,Duration
0,IndiGo,Banglore,New Delhi,22:20,01:10 22 Mar,0,3897,24,3,2019,6,1,12,2.833333
1,Air India,Kolkata,Banglore,05:50,13:15,2,7662,5,1,2019,5,0,1,7.416667
2,Jet Airways,Delhi,Cochin,09:25,04:25 10 Jun,2,13882,6,9,2019,4,0,36,19.000000
3,IndiGo,Kolkata,Banglore,18:05,23:30,1,6218,5,12,2019,3,0,49,5.416667
4,IndiGo,Banglore,New Delhi,16:50,21:35,1,13302,3,1,2019,3,0,1,4.750000


In [25]:
foo['Dep_Time']=pd.to_datetime(foo['Dep_Time'],format='%H:%M').dt.time
foo.dtypes

Airline             object
Source              object
Destination         object
Dep_Time            object
Arrival_Time        object
Total_Stops          int64
Price                int64
Journey Date         int64
Journey Month        int64
Journey Year         int64
Journey Day          int64
Journey Weekend      int64
Journey Week         int64
Duration           float64
hi                   int64
dtype: object

In [26]:
foo['hi']=foo['Dep_Time'].dt.hour

AttributeError: Can only use .dt accessor with datetimelike values

In [24]:
foo.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Price,Journey Date,Journey Month,Journey Year,Journey Day,Journey Weekend,Journey Week,Duration,hi
0,IndiGo,Banglore,New Delhi,1900-01-01 22:20:00,01:10 22 Mar,0,3897,24,3,2019,6,1,12,2.833333,22
1,Air India,Kolkata,Banglore,1900-01-01 05:50:00,13:15,2,7662,5,1,2019,5,0,1,7.416667,5
2,Jet Airways,Delhi,Cochin,1900-01-01 09:25:00,04:25 10 Jun,2,13882,6,9,2019,4,0,36,19.000000,9
3,IndiGo,Kolkata,Banglore,1900-01-01 18:05:00,23:30,1,6218,5,12,2019,3,0,49,5.416667,18
4,IndiGo,Banglore,New Delhi,1900-01-01 16:50:00,21:35,1,13302,3,1,2019,3,0,1,4.750000,16


In [19]:
train_df[['Date_of_Journey']]=pd.to_datetime(train_df['Date_of_Journey'])
train_df['Journey Day']=train_df['Date_of_Journey'].dt.day
train_df['Journey Month']=train_df['Date_of_Journey'].dt.month
train_df['Journey Year']=train_df['Date_of_Journey'].dt.year
train_df['Holiday']=

In [20]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey Day,Journey Month,Journey Year
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,5,1,2019
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,6,9,2019
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,5,12,2019
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,3,1,2019


In [88]:
train.drop(['Date_of_Journey'],axis=1,inplace=True)

In [89]:
train.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey Day,Journey Month,Journey Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,5,1,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,6,9,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,5,12,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,3,1,2019


In [90]:
# SInce the difference of dep_time and arrival_time is same as the duration, we will keep that and remove the other two

train.drop(['Dep_Time','Arrival_Time'],axis=1,inplace=True)
train.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Journey Day,Journey Month,Journey Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,5,1,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,6,9,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,5,12,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,3,1,2019


In [91]:
duration = list(train.Duration)
for i in range(len(duration)):
    if len(duration[i].split())!=2:
        if 'h' in duration[i]:
            duration[i] = duration[i].strip()+' 0m'
        else:
            duration[i] = '0h ' + duration[i].strip()

duration_hour =[]
duration_min = []

for i in range(len(duration)):
    duration_hour.append(duration[i].split(sep='h')[0])
    duration_min.append(duration[i].split(sep='m')[0].split()[-1])

In [92]:
train['Duration_hour'] = duration_hour
train['Duration_min'] = duration_min
train.drop(['Duration'],axis=1,inplace=True)

train.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey Day,Journey Month,Journey Year,Duration_hour,Duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,2019,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,5,1,2019,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,6,9,2019,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,5,12,2019,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,3,1,2019,4,45


In [93]:
train['Duration_min']=train['Duration_min'].astype(float)/60
train['Duration']=train['Duration_hour'].astype(float)+train['Duration_min']
train.drop(['Duration_hour','Duration_min'],axis=1,inplace=True)
train.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey Day,Journey Month,Journey Year,Duration
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,2019,2.833333
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,5,1,2019,7.416667
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,6,9,2019,19.000000
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,5,12,2019,5.416667
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,3,1,2019,4.750000


In [94]:
(train['Additional_Info'].value_counts()/train.shape[0])*100
#since approx 80% of the additional info has no info so we can remvoe it 

train.drop(['Additional_Info'],axis=1,inplace=True)

In [95]:
train.head()

,Airline,Source,Destination,Route,Total_Stops,Price,Journey Day,Journey Month,Journey Year,Duration
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,3897,24,3,2019,2.833333
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,7662,5,1,2019,7.416667
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,13882,6,9,2019,19.000000
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,6218,5,12,2019,5.416667
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,13302,3,1,2019,4.750000


In [96]:
train['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [97]:
stops = {'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}

In [98]:
train['Stops']=(train['Total_Stops'].map(stops))

In [99]:
train['Stops'].unique()

array([ 0.,  2.,  1.,  3., nan,  4.])

In [100]:
train.head()

,Airline,Source,Destination,Route,Total_Stops,Price,Journey Day,Journey Month,Journey Year,Duration,Stops
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,3897,24,3,2019,2.833333,0.0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,7662,5,1,2019,7.416667,2.0
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,13882,6,9,2019,19.000000,2.0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,6218,5,12,2019,5.416667,1.0
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,13302,3,1,2019,4.750000,1.0


In [103]:
train.drop(['Total_Stops'],axis=1,inplace=True)

In [104]:
train.head()

,Airline,Source,Destination,Price,Journey Day,Journey Month,Journey Year,Duration,Stops
0,IndiGo,Banglore,New Delhi,3897,24,3,2019,2.833333,0.0
1,Air India,Kolkata,Banglore,7662,5,1,2019,7.416667,2.0
2,Jet Airways,Delhi,Cochin,13882,6,9,2019,19.000000,2.0
3,IndiGo,Kolkata,Banglore,6218,5,12,2019,5.416667,1.0
4,IndiGo,Banglore,New Delhi,13302,3,1,2019,4.750000,1.0


In [9]:
list(train.Airline.unique())

['IndiGo',
 'Air India',
 'Jet Airways',
 'SpiceJet',
 'Multiple carriers',
 'GoAir',
 'Vistara',
 'Air Asia',
 'Vistara Premium economy',
 'Jet Airways Business',
 'Multiple carriers Premium economy',
 'Trujet']